In [1]:
import pandas as pd
df = pd.DataFrame(columns=['名字','简介','粉丝数','点赞数'])
###创建数据存储文件

In [26]:
import requests
import json

headers= {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7", 
    "Accept-Encoding": "gzip, deflate, br", 
    "Accept-Language": "zh-CN,zh;q=0.9", 
    "Cache-Control": "max-age=0",
    "Sec-Fetch-Dest": "document",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Site": "none",
    "Sec-Fetch-User": "?1",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"}
###用这个header可以解决获取不到的问题

from bs4 import BeautifulSoup
import time
def analyze_upspace(id: int):
    fan_num = 0 ###粉丝数
    good_num = 0 ###点赞数
    id = str(id)
    stat =  'https://api.bilibili.com/x/relation/stat?vmid=' + id
    upstat =  'https://api.bilibili.com/x/space/upstat?mid=' + id
    stat = requests.get(stat, headers=headers)
    stat = json.loads(stat.text)
    if(stat['data']):
        stat = stat['data']
        fan_num = int(stat['follower'])
    upstat = requests.get(upstat, headers=headers)
    print(upstat.text)
    upstat = json.loads(upstat.text)
    if(upstat['data']):
        upstat = upstat['data']
        good_num = int(upstat['likes'])
    ### 这里可能会崩掉，可能是服务器的问题

    ### stat['following'] - 关注数
    # upstat['archive']['view'] - 视频播放量
    # upstat['article']['view'] - 文章浏览量

    #name = "" ###名字，用来比较确认(舍弃)
    #info =  'https://api.bilibili.com/x/space/wbi/acc/info?mid=' + id + '&token=&platform=web&web_location=1550101'
    #info = requests.get(info)
    #info = json.loads(info.text)
    #print(info)
    #info = info['data']
    # print(info)
    #name = info['name']
    ### 读取姓名的链接失效了...
    return fan_num, good_num

In [27]:
analyze_upspace(333777)

{"code":0,"message":"0","ttl":1,"data":{}}


(4, 0)

In [3]:
top_up = 'https://api.bilibili.com/x/activity/operation/list?pn=1&ps=50&source_id=63a14c147e8469700e19a45d'
top_up = requests.get(top_up)
top_up = json.loads(top_up.text)
informations = top_up['data']['list']
for info in informations:
    id = int(info['object']['mid'])
    name = info['object']['name']
    desc = info["properties"]['desc']
    fan_num, good_num = analyze_upspace(id)
    df = df.append({'名字': name,'简介': desc,'粉丝数': fan_num, '点赞数': good_num},ignore_index=True)

In [4]:
df.to_csv("data.csv")

In [ ]:
def search_bilibili_by_name(name: str):
    fan_num = 0 ###粉丝数
    good_num = 0 ###点赞数
    url = 'https://search.bilibili.com/upuser?keyword=' + name
    response = requests.get(url)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    related_informations = soup.find_all('div', {'class' : "col_6 col_md_4 mb_x60"})
    found = False
    for information in related_informations:
        space_link = information.find('a', {'class', 'mr_md'})
        space_link = space_link.get('href')
        found = True
        break
    if(not found):
        print(f"没有找到名字为{name}的UP主")
    return fan_num, good_num
### 这个方法有点问题，b站采用异步查询的方法，用这个方法耗时太长

In [4]:
### 这里采用的思想是希望通过模拟浏览器减缓异步，目前舍弃(无法找到id)
from selenium import webdriver
from selenium.webdriver.chromium import service
driver = webdriver.Chrome(r"C:/Program Files/Google/Chrome/Application/chromedriver.exe")

'''
第一步: 获取所有百大UP主的名字
'''
url = 'https://www.bilibili.com/BPU2022#/poweruplist'
driver.get(url)

import time
time.sleep(6)###由于采用了动态读取，因此必须给浏览器足够时间来缓存

html = driver.page_source

from selenium.webdriver.common.action_chains import ActionChains


soup = BeautifulSoup(html, 'lxml')
div = soup.find('div', {'id': 'app'}).find('div', {'class': 'power-up-total-list'}).find('div', {'class': 'main-content'}).find('div', {'class': 'list'})
lis = div.find_all('li', {'class': 'f-sb-s'})
for li in lis:
    name = li.find('p', {'class':'name'})
    desc = li.find('p', {'class':'desc'})
    name = name.text
    desc = desc.text
    df = df.append({'名字': name, '简介': desc ,'粉丝数': "-1", '点赞数': "-1"}, ignore_index=True)

C:\Windows\Temp\ipykernel_19832\3001865980.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r"C:/Program Files/Google/Chrome/Application/chromedriver.exe")


In [5]:
df.to_csv("data.csv")